In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

/Users/being/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6], device='mps:0')


In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [10]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 1.7365e-01, -9.4056e-02,  9.6688e-02, -4.6166e-02, -4.7134e-01,
          6.4820e-02, -4.2383e-01,  5.2967e-01, -2.9196e-01, -3.1886e-01,
         -6.3052e-03,  9.2570e-02,  1.1488e-01, -1.3141e-01, -4.5792e-01,
         -1.0650e-01,  2.1878e-01,  3.7785e-01,  4.6147e-01,  2.2805e-01],
        [-3.4010e-01,  1.5772e-01, -6.5096e-02, -5.7976e-01, -2.2711e-01,
         -2.4334e-02, -3.6482e-04,  2.3724e-01, -1.2539e-01, -2.4084e-01,
         -2.0818e-01,  2.2706e-01, -8.8293e-02, -2.3455e-01, -4.3897e-01,
         -1.0921e-01,  6.8531e-02,  2.7857e-01,  4.5826e-01,  4.9753e-01],
        [-8.5127e-02,  2.9652e-01,  2.1957e-01,  1.5393e-01, -6.5889e-01,
          1.8038e-01, -1.5100e-01,  5.5244e-01,  1.0626e-02, -7.1278e-01,
         -1.4246e-01,  7.3206e-02, -1.7311e-01, -3.8047e-01, -6.3260e-01,
         -5.0031e-02,  4.3692e-01,  4.6811e-01,  7.1301e-01,  3.3493e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1736, 0.0000, 0.0967, 0.0000, 0.0000,

In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [13]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [14]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0107,  0.0125, -0.0260,  ..., -0.0320,  0.0155, -0.0285],
        [-0.0037, -0.0126, -0.0155,  ...,  0.0270,  0.0341,  0.0078]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0298, 0.0065], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0429, -0.0138,  0.0289,  ..., -0.0225, -0.0228,  0.0039],
        [ 0.0070, -0.0366,  0.0035,  ...,  0.0232,  0.0135, -0.0344]],
       device='mps:0', grad_fn=<SliceBa